# Setup

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import pytest as pt

from datetime import datetime

In [ ]:
# set the file name. This needs to be done manually, as Jupyter makes it oddly difficult to query the current notebook filename.
__file__ = 'comparer.ipynb'

import ipytest.magics
import pytest

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
%%HTML
<style>
    .container { width:100% !important; } 
</style>

In [ ]:
# matplotlib configuration
%matplotlib inline
matplotlib.style.use('bmh')

# Common Plotly configuration
py.init_notebook_mode(connected=True)
config={
    'showLink': False,
    'modeBarButtonsToRemove': ['sendDataToCloud', 'hoverClosestCartesian', 'hoverCompareCartesian'],
}

In [ ]:
def line_chart(y, x=None):
    py.iplot([{'y':y, 'x':x}], config=config)

# Assumptions

In [ ]:
cpi = 0.02
km_per_year = 15000
cost_per_tyre = 350
tyre_life_km = 45000
fuel_cost_per_litre = 1.50
fuel_efficiency_litres_per_100km = 7.0

purchase_price = 35000
age_at_purchase = 0
years_to_model = 15

# Standing Costs

In [ ]:
insurance_per_year = 500
registration_per_year = 500
service_cost = 400
service_interval_km = 10000
service_interval_years = 1.0
roadside_assist_per_year = 200

# Depreciation

In [ ]:
def depreciation_rate_decelerating(age_in_years):
    '''Go with the simple assumption of 15% for first 3 years, and 10% after that.
       This might be too aggressive, but can fine-tune later.
    '''
    if age_in_years < 3:
        return 0.15
    else:
        return 0.10
    
class FlatRate():
    def __init__(self, rate=0.1):
        self.rate = rate
        
    def __call__(self, year):
        return self.rate

In [ ]:
def calc_depreciation(
    initial_value,
    years=10,
    initial_age=0,
    dep_rate_func=FlatRate()):
    """ Calculates the yearly depreciated value for a range of years, 
        given an initial value, the number of years, 
        and a function that gives the depreciation rate for a given year.
        
        Args:
            initial_value (double): the starting value.
            years (int): the number of years to calculate.
            initial_age(int): vehicle age at start 
            dep_rate_func (callable): Function accepting a single int that 
                returns the depreciation rate to use for the given age in years.
            
        Returns:
            dep_value: numpy array containing the depreciated value for each year.
            yearly_loss: numpy array containing the depreciation loss for 
                each year, defined as the difference in value between the 
                start and end of the year.
    """
    
    dep_value = np.zeros(years)
    yearly_loss = np.zeros(years)
    dep_value[0] = initial_value
    previous = initial_value
    for yr in range(1, years):
        rate = dep_rate_func(initial_age + yr - 1)
        previous *= (1.0 - rate)
        dep_value[yr] = previous
        
    for yr in range(years):
        rate = dep_rate_func(initial_age + yr)
        yearly_loss[yr] = rate * dep_value[yr]
        
    return dep_value, yearly_loss

## Tests

In [ ]:
%%run_pytest[clean] -v --tb=line

def decelerating(y):
    if y < 3:
        return 0.15
    elif y < 5:
        return 0.1
    else:
        return 0.08
    
flat_rate_15 = FlatRate(0.15)
    
def test_depreciated_value_result_sizes():
    iv = 100  # initial value
    y = 5  # num years
    ia = 0  # initial age
    dv, loss = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=flat_rate_15)
    assert len(dv) == y
    assert len(loss) == y
    
def test_depreciated_value_first_year():
    iv = 100
    y = 5
    ia = 0  # initial age
    dv, loss = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=flat_rate_15)
    assert dv[0] == pt.approx(iv)
    
def test_depreciated_value_second_year():
    iv = 1000
    y = 5
    ia = 0  # initial age
    dv, _ = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=flat_rate_15)
    assert dv[1] == pt.approx(iv * (1 - flat_rate_15(0)))
    
def test_depreciated_value():
    iv = 100
    y = 5
    ia = 0
    dv, _ = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=flat_rate_15)
    r = 1 - flat_rate_15(0)
    assert dv[0] == pt.approx(iv)
    assert dv[1] == pt.approx(dv[0] * r)
    assert dv[2] == pt.approx(dv[1] * r)
    assert dv[3] == pt.approx(dv[2] * r)
    assert dv[4] == pt.approx(dv[3] * r)
    
def test_loss_vs_dep_value():
    iv = 100
    y = 5
    ia = 0
    dv, loss = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=decelerating)
    for i in range(y):
        assert loss[i] == pt.approx(decelerating(i) * dv[i])
    
def test_cumulative_loss():
    iv = 15000
    y = 10
    ia = 0
    dv, loss = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=decelerating)
    assert dv[0] - dv[-1] == pt.approx(loss[0:-1].sum())
    

# Compound Interest
Solves $A = P(1+\frac{r}{n})^{nt}$ for
* P = principal
* r = annual rate
* n = number of times interest is compounded per year
* t = number of years to compound

Returns an array giving the new principal for each year in the sequence.

In [ ]:
def compound_interest(
    principal,
    annual_rate,
    years,
    compounds_per_year=1,
):
    return np.array([principal * ((1 + annual_rate/compounds_per_year)**(yr*compounds_per_year)) for yr in range(years)])

In [ ]:
%%run_pytest[clean] -v --tb=line

def test_compound_interest_num_results():
    p = compound_interest(8000, 0.073, 10, 1)
    assert 10 == len(p)
    
def test_compound_interest():
    actual = compound_interest(8000, 0.073, 5, 1)
    expected = [8000.0, 8584.0, 9210.632, 9883.008, 10604.468]
    np.testing.assert_allclose(actual, expected, verbose=True)
    

# Fuel Costs

In [ ]:
cpi = 0.02
km_per_year = 15000
cost_per_tyre = 350
tyre_life_km = 45000
fuel_cost_per_litre = 1.50
fuel_efficiency_litres_per_100km = 7.0

In [ ]:
def fuel_used(distance_km, litres_per_100km):
    return distance_km * litres_per_100km / 100.0

In [ ]:
def yearly_fuel_cost(km_per_year, fuel_efficiency, years, cpi, initial_fuel_cost):
    assert cpi >= 0
    assert cpi <= 1.0
    
    fuel_used_per_year = fuel_used(km_per_year, fuel_efficiency)
    indexed_fuel_cost_per_litre = compound_interest(principal=initial_fuel_cost, annual_rate=cpi, years=years, compounds_per_year=1)
    return indexed_fuel_cost_per_litre * fuel_used_per_year

In [ ]:
%%run_pytest[clean] -v --tb=line

def test_fuel_used_100km():
    assert fuel_used(100, 8) == 8.0
    
def test_fuel_used_15000km():
    assert fuel_used(15000, 15.8) == 2370.0
    
def test_fuel_cost_over_10years():
    expected = [1890.0, 1929.69, 1970.214, 2011.588, 2053.831, 2096.962, 2140.998, 2185.959, 2231.864, 2278.733]
    actual = yearly_fuel_cost(km_per_year=15000, fuel_efficiency=8.4, years=10, cpi=2.1/100.0, initial_fuel_cost=1.5)
    np.testing.assert_allclose(actual, expected, atol=0.001)